In [22]:

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2TokenizerFast, GPT2ForTokenClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings
from peft import LoraConfig, get_peft_model

warnings.filterwarnings('ignore')


In [31]:

# Stałe
BATCH_SIZE = 16
EPOCHS = 500
LEARNING_RATE = 1e-5
MAX_LEN = 128
TRAIN_SIZE = 0.8


In [24]:

# Wczytanie i przygotowanie danych
df = pd.read_csv('data/annotations_all_batches - WORD - SECOND BATCH.csv')
df = df.fillna(method='ffill')

# Grupowanie po sentence_id
sentences = df.groupby('sentence_id').agg({
    'word': lambda x: list(x),
    'final-annotation': lambda x: list(x)
}).reset_index()


In [25]:

# Konfiguracja modelu GPT2 z adapterem PEFT
model_name = "sdadas/polish-gpt2-small"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name, add_prefix_space=True)
tokenizer.pad_token = tokenizer.eos_token  # Ustawienie tokenu paddingu na <eos>

# Ładowanie modelu GPT2
base_model = GPT2ForTokenClassification.from_pretrained(model_name, num_labels=4)

# Konfiguracja adaptera PEFT (LoRA)
peft_config = LoraConfig(
    r=8,  # Rank
    lora_alpha=32,
    target_modules=["c_attn"],  # Warstwy do zastosowania LoRA
    lora_dropout=0.1,
    bias="none"
)

# Dodanie PEFT do modelu
model = get_peft_model(base_model, peft_config)
print("Model GPT2 z adapterem PEFT gotowy!")


Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at sdadas/polish-gpt2-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model GPT2 z adapterem PEFT gotowy!


In [26]:

# Klasa dataset
class TokenClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        words = self.texts[idx]
        labels = self.labels[idx]
        
        # Tokenizacja
        encoding = self.tokenizer(
            words,
            is_split_into_words=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Dostosowanie etykiet do tokenów
        word_ids = encoding.word_ids()
        label_ids = []
        
        for word_id in word_ids:
            if word_id is None or word_id == tokenizer.eos_token_id:
                label_ids.append(-100)
            else:
                label_ids.append(labels[word_id])
                
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label_ids)
        }


In [27]:

# Funkcja treningu
def train(model, data_loader, optimizer, device):
    model = model.train()
    total_loss = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    return total_loss / len(data_loader)


In [28]:

# Podział danych na treningowe i walidacyjne
train_texts, val_texts, train_labels, val_labels = train_test_split(
    sentences['word'].values, sentences['final-annotation'].values, train_size=TRAIN_SIZE
)

# Tworzenie datasetów i data loaderów
train_dataset = TokenClassificationDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = TokenClassificationDataset(val_texts, val_labels, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


In [35]:

# Trenowanie modelu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)


In [40]:

for epoch in range(10000):
    avg_loss = train(model, train_loader, optimizer, device)
    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {avg_loss:.4f}")


Epoch 1/500, Loss: 0.1449
Epoch 2/500, Loss: 0.1523
Epoch 3/500, Loss: 0.1231
Epoch 4/500, Loss: 0.1182
Epoch 5/500, Loss: 0.1236
Epoch 6/500, Loss: 0.1276
Epoch 7/500, Loss: 0.1265
Epoch 8/500, Loss: 0.1134
Epoch 9/500, Loss: 0.1057
Epoch 10/500, Loss: 0.1488
Epoch 11/500, Loss: 0.1155
Epoch 12/500, Loss: 0.1401
Epoch 13/500, Loss: 0.1304
Epoch 14/500, Loss: 0.1275
Epoch 15/500, Loss: 0.1427
Epoch 16/500, Loss: 0.1438
Epoch 17/500, Loss: 0.1235
Epoch 18/500, Loss: 0.1296
Epoch 19/500, Loss: 0.1295
Epoch 20/500, Loss: 0.1306
Epoch 21/500, Loss: 0.1136
Epoch 22/500, Loss: 0.1321
Epoch 23/500, Loss: 0.1265
Epoch 24/500, Loss: 0.1020
Epoch 25/500, Loss: 0.1233
Epoch 26/500, Loss: 0.1284
Epoch 27/500, Loss: 0.1287
Epoch 28/500, Loss: 0.1256
Epoch 29/500, Loss: 0.1318
Epoch 30/500, Loss: 0.1127
Epoch 31/500, Loss: 0.1230
Epoch 32/500, Loss: 0.1134
Epoch 33/500, Loss: 0.1302
Epoch 34/500, Loss: 0.1437
Epoch 35/500, Loss: 0.1126
Epoch 36/500, Loss: 0.1237
Epoch 37/500, Loss: 0.1070
Epoch 38/5

KeyboardInterrupt: 

In [41]:

# Ewaluacja modelu
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        predictions.extend(preds.cpu().numpy().tolist())
        true_labels.extend(labels.cpu().numpy().tolist())

# Raport klasyfikacji
print(classification_report(np.concatenate(true_labels), np.concatenate(predictions)))


              precision    recall  f1-score   support

        -100       0.00      0.00      0.00        59
           0       0.50      1.00      0.67         2
           1       0.28      0.87      0.42        31
           2       0.78      0.37      0.50        19
           3       0.78      0.82      0.80        17

    accuracy                           0.39       128
   macro avg       0.47      0.61      0.48       128
weighted avg       0.29      0.39      0.29       128



In [42]:
label_mapping = {0: 'negatywny', 1: 'neutralny', 2: 'pozytywny', 3: 'inne'}

def predict_sentence_gpt2(sentence_words):
    model.eval()
    with torch.no_grad():
        # Tokenizacja zdania
        inputs = tokenizer(
            sentence_words,
            is_split_into_words=True,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=MAX_LEN
        ).to(device)
        
        # Przewidywania modelu
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=2)

        word_predictions = []
        tokenized_sentence = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        
        current_word = ""
        current_predictions = []
        
        for token, pred in zip(tokenized_sentence, predictions[0]):
            # Pomijanie tokenów paddingu oraz <eos>
            if token in [tokenizer.pad_token, tokenizer.eos_token]:
                continue
            
            # Jeśli token zaczyna się od "Ġ", oznacza to początek nowego słowa (GPT2 używa "Ġ" jako prefiksu słowa)
            if token.startswith("Ġ"):
                if current_predictions:
                    # Wybieramy najczęstszą predykcję dla bieżącego słowa
                    word_predictions.append(max(set(current_predictions), key=current_predictions.count))
                current_predictions = []
            
            # Dodajemy predykcję do bieżącej listy
            current_predictions.append(pred.item())
        
        # Dodaj ostatnie słowo
        if current_predictions:
            word_predictions.append(max(set(current_predictions), key=current_predictions.count))
        
        return word_predictions

# Przykładowe predykcje
print("\nPrzykładowe predykcje dla zdań ze zbioru testowego:")
for i in range(min(3, len(val_texts))):
    sentence = val_texts[i]
    predictions = predict_sentence_gpt2(sentence)
    
    print(f"\nZdanie {i + 1}:")
    for word, pred in zip(sentence, predictions):
        pred_label = label_mapping[pred]
        print(f"Słowo: {word:15} Predykcja: {pred_label}")



Przykładowe predykcje dla zdań ze zbioru testowego:

Zdanie 1:
Słowo: Do              Predykcja: neutralny
Słowo: Bosch           Predykcja: neutralny
Słowo: SMV53L10EU      Predykcja: neutralny
Słowo: pasuje          Predykcja: pozytywny
Słowo: IDEALNIE        Predykcja: neutralny
Słowo: wpasowuje       Predykcja: neutralny
Słowo: się             Predykcja: neutralny
Słowo: w               Predykcja: inne
Słowo: nowe            Predykcja: neutralny
Słowo: miejsce         Predykcja: neutralny
Słowo: które           Predykcja: inne
Słowo: jest            Predykcja: inne
Słowo: przygotowane    Predykcja: neutralny
Słowo: w               Predykcja: inne
Słowo: zmywarce        Predykcja: neutralny
Słowo: więc            Predykcja: inne
Słowo: jeśli           Predykcja: inne
Słowo: po              Predykcja: inne
Słowo: drodze          Predykcja: neutralny
Słowo: coś             Predykcja: neutralny
Słowo: się             Predykcja: inne
Słowo: połamało        Predykcja: negatywny
Słowo: w

In [45]:
# Ścieżki do zapisu
model_save_path = "gpt2_model/gpt2_with_peft_model"
tokenizer_save_path = "gpt2_model/gpt2_tokenizer"

# Zapis modelu
model.save_pretrained(model_save_path)
print(f"Model zapisano w: {model_save_path}")

# Zapis tokenizera
tokenizer.save_pretrained(tokenizer_save_path)
print(f"Tokenizer zapisano w: {tokenizer_save_path}")


Model zapisano w: gpt2_model/gpt2_with_peft_model
Tokenizer zapisano w: gpt2_model/gpt2_tokenizer
